# グローバーのアルゴリズムを使って、データベース探索を行う

ここでは、グローバーのアルゴリズムの紹介と、そのアルゴリズムを使用して構造化されていないデータベースを検索する問題を考えます。アルゴリズムを説明した後に、Qiskitを使用してグローバーのアルゴリズムを実装します。

## 目次

1. [はじめに](#introduction)
2. [例: 3量子ビットの場合](#3qubits)
3. [演習](#problems)


## 1. はじめに <a id='introduction'></a>

量子コンピュータが古典コンピュータの計算能力を優位に上回る性能を発揮するためには、量子計算の特徴をうまく活用するアルゴリズムを考える必要があります。そのような量子アルゴリズムの一つとして知られているのが、グローバーのアルゴリズムです。このアルゴリズムは構造化されていないデータベースの検索に適しており、古典計算より少ない計算量で答えを得られることが証明されています。このアルゴリズムは振幅増幅と呼ばれる手法をベースにしており、量子アルゴリズムのサブルーチンとしても幅広く活用されている。


## 非構造化データの検索

$N$個の要素からなるリストがあり、その中の一つの要素$w$を見つけたいとします。
求める要素$w$を見つけるためには、古典計算では最悪の場合$N$回、平均では$N/2$回リストを調べる必要があります。グローバーのアルゴリズムでは、おおよそ$\sqrt{N}$回の検索で$w$を見つけることができることが知られています。つまり、古典計算に対して二次の高速化が可能というわけです。

## グローバーのアルゴリズム

ここでは$n$個の量子ビットを考え、その量子ビットが存在する可能性のある全ての計算基底でリストが構成されているものとします。つまり$N=2^n$として、リストは$|00\cdots00\rangle$, $|00\cdots01\rangle$, $|00\cdots10\rangle$, $\cdots$, $|11\cdots11\rangle$までの$N$個の要素（10進数表記で$|0\rangle$, $|1\rangle$, $\cdots$, $|N-1\rangle$）を含んでいます。

### 位相オラクル

まず、位相オラクル$U|x\rangle=(-1)^{f(x)}|x\rangle$を考える。この場合、関数$f(x)$として
$
f(x) = \bigg\{
\begin{aligned}
&1 \quad \text{if} \; x = w \\
&0 \quad \text{else} \\
\end{aligned}
$
を考えると、求める解$w$の位相を反転するオラクル$U_w$
$$
U_w:\begin{aligned}
&|w\rangle \to -|w\rangle\\
&|x\rangle \to |x\rangle \quad \forall \; x \neq w
\end{aligned}
$$
が得られます。この時、$U_w$は$U_w=I-2|w\rangle\langle w|$と与えられる。また、
関数$f_0(x)$として
$
f_0(x) = \bigg\{
\begin{aligned}
&0 \quad \text{if} \; x = 0 \\
&1 \quad \text{else} \\
\end{aligned}
$
を考えると、0以外の位相を反転するユニタリー$U_0$
$$
U_0:\begin{aligned}
&|0\rangle^{\otimes n} \to |0\rangle^{\otimes n}\\
&|x\rangle \to -|x\rangle \quad \forall \; x \neq 0
\end{aligned}
$$
も得ることができます。この時、$U_0$は$U_0=2|0\rangle\langle 0|^{\otimes n}-I$になる。

### グローバーの量子回路

<img src="figs/grover.png" width=70%>
G
<img src="figs/grover_iter.png" width=50%>


<img src="figs/grover_iter1.png" width=70%>

$$
|s\rangle = H^{\otimes n}|0\rangle^{\otimes n} = \frac{1}{\sqrt{N}}\sum_{x=0}^{N-1}|x\rangle
$$

$|s\rangle$に含まれている$|w\rangle$の直交状態を$|w^{\perp}\rangle$とすると、$|w^{\perp}\rangle:=\frac{1}{\sqrt{N-1}}\sum_{x \neq w}|x\rangle$であり、($|w\rangle$, $|w^{\perp}\rangle$)が張る2次元平面では、$|w^{\perp}\rangle=\begin{bmatrix}1\\0\end{bmatrix}$と$|w\rangle=\begin{bmatrix}0\\1\end{bmatrix}$である。

$$
\begin{aligned}
|s\rangle&=\sqrt{\frac{N-1}{N}}|w^{\perp}\rangle+\sqrt{\frac1N}|w\rangle\\
&=: \cos\frac\theta2|w^{\perp}\rangle+\sin\frac\theta2|w\rangle\\
&= \begin{bmatrix}\cos\frac\theta2\\\sin\frac\theta2\end{bmatrix}
\end{aligned}
$$
$\theta=2\arcsin\frac{1}{\sqrt{N}}$

<img src="figs/grover_rot1.png" width=30%>
<img src="figs/grover_rot2.png" width=30%>

$U_w=I-2|w\rangle\langle w|=\begin{bmatrix}1&0\\0&-1\end{bmatrix}$

$U_0=2|0\rangle\langle0|^{\otimes n}-I$
$$
\begin{aligned}
H^{\otimes n}U_0H^{\otimes n}&=2|s\rangle\langle s|-I\\
&= \begin{bmatrix}\cos\theta&\sin\theta\\\sin\theta&-\cos\theta\end{bmatrix}
\end{aligned}
$$
<img src="figs/grover_rot3.png" width=30%>
<img src="figs/grover_rot4.png" width=30%>


$$
\begin{aligned}
G&=(2|s\rangle\langle s|-I)U_w\\
&= \begin{bmatrix}\cos\theta&-\sin\theta\\\sin\theta&\cos\theta\end{bmatrix}
\end{aligned}
$$
$G$を$r$回繰り返した後の状態は
$
G^r|s\rangle=\begin{bmatrix}\cos\frac{2r+1}{2}\theta\\\sin\frac{2r+1}{2}\theta\end{bmatrix}
$

<img src="figs/grover_amp.png" width=90%>


$\frac{2r+1}2\theta\approx\frac{\pi}2$となる$r$回だけ回転すれば良い。
$\sin\frac\theta2=\frac{1}{\sqrt{N}}\approx\frac\theta2$とすると、$r\approx\frac\pi4\sqrt{N}$

$|\psi\rangle:=\sum_k a_k|k\rangle$とすると、
$$
\begin{aligned}
(2|s\rangle\langle s|-I)|\psi\rangle&=\frac2N\sum_i|i\rangle\cdot\sum_{j,k}a_k\langle j|k\rangle-\sum_k a_k|k\rangle\\
&= 2\frac{\sum_i a_i}{N}\sum_k|k\rangle-\sum_k a_k|k\rangle\\
&= \sum_k(2\langle a\rangle-a_k)|k\rangle
\end{aligned}
$$
$\langle a\rangle\equiv\frac{\sum_i a_i}{N}$

$a_k=\langle a\rangle-\Delta$とすると$2\langle a\rangle-a_k=\langle a\rangle+\Delta$になり、$\langle a\rangle$に関する反転を表している。

複数のデータを探索する場合（例えば、$N=2^m$個のデータから、$M$個のデータ$w$を探す）、同様の議論ができる。

$$
\begin{aligned}
&|w\rangle:=\frac{1}{\sqrt{M}}\sum_{i=0}^{M-1}|w_i\rangle\\
&|w^{\perp}\rangle:=\frac{1}{\sqrt{N-M}}\sum_{x\notin\{w_0,\cdots,w_{M-1}\}}|x\rangle
\end{aligned}
$$

$$
\begin{aligned}
|s\rangle&=\sqrt{\frac{N-M}{N}}|w^{\perp}\rangle+\sqrt{\frac{M}{N}}|w\rangle\\
&=: \cos\frac\theta2|w^{\perp}\rangle+\sin\frac\theta2|w\rangle\\
\end{aligned}
$$
$\theta=2\arcsin\sqrt{\frac{M}{N}}$

$r\approx\frac\pi4\sqrt{\frac{N}{M}}$